In [ ]:
import os
import sys
import subprocess
import requests
from pathlib import Path
import zipfile
from tqdm import tqdm
import pkg_resources

def check_versions():
    python_version = sys.version_info
    if python_version.major == 3 and python_version.minor >= 12:
        print(f"Python {python_version.major}.{python_version.minor} detected. Attempting to setup Python 3.11 environment...")
        try:
            # Create virtual environment with Python 3.11
            commands = [
                "conda create -n comfy_env python=3.11 -y",
                "conda activate comfy_env"
            ]
            for cmd in commands:
                output, error, rc = run_command(cmd)
                if rc != 0:
                    raise Exception(f"Failed to setup Python 3.11: {error}")
            
            print("Successfully switched to Python 3.11")
            # Re-run this script in the new environment
            os.execv(sys.executable, ['python'] + sys.argv)
        except Exception as e:
            print(f"Error setting up Python 3.11: {e}")
            print("Please manually install Python 3.11 and try again")
            sys.exit(1)

def run_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    output, error = process.communicate()
    return output, error, process.returncode

def download_file(url, destination, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 KB
            
            with open(destination, 'wb') as file, tqdm(
                desc=f"Attempt {attempt + 1}: {os.path.basename(destination)}",
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(block_size):
                    size = file.write(data)
                    progress_bar.update(size)
            return True
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries - 1:
                raise e
    return False

def setup_directories():
    base_path = '/workspace/ComfyUI'
    directories = [
        'models/unet',
        'models/clip',
        'models/vae',
        'models/controlnet/flux',
        'models/upscale_models',
        'models/pulid',
        'models/insightface/models/antelopev2',
        'models/lora'
    ]
    
    for dir in directories:
        Path(f"{base_path}/{dir}").mkdir(parents=True, exist_ok=True)

def install_dependencies():
    commands = [
        "python -m pip install --use-pep517 facexlib",
        "python -m pip install onnxruntime"
    ]
    
    for cmd in commands:
        output, error, rc = run_command(cmd)
        if rc != 0:
            print(f"Error running {cmd}: {error}")

def download_models():
    models = {
        'models/unet/flux1-dev-fp8-e4m3fn.safetensors': 
            'https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8-e4m3fn.safetensors',
        'models/clip/clip_l.safetensors':
            'https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors',
        'models/clip/t5xxl_fp8_e4m3fn.safetensors':
            'https://huggingface.co/mcmonkey/google_t5-v1_1-xxl_encoderonly/resolve/main/t5xxl_fp8_e4m3fn.safetensors',
        'models/vae/ae.safetensors':
            'https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors',
        'models/controlnet/flux/diffusion_pytorch_model.safetensors':
            'https://huggingface.co/InstantX/FLUX.1-dev-Controlnet-Union/resolve/main/diffusion_pytorch_model.safetensors',
        'models/upscale_models/4x-ClearRealityV1.pth':
            'https://huggingface.co/agsz5/4x_clr_rlty/resolve/main/4x-ClearRealityV1.pth',
        'models/pulid/pulid_flux_v0.9.0.safetensors':
            'https://huggingface.co/guozinan/PuLID/resolve/main/pulid_flux_v0.9.0.safetensors',
        'models/lora/8-step-flux.safetensors':
            'https://huggingface.co/alimama-creative/FLUX.1-Turbo-Alpha/resolve/main/diffusion_pytorch_model.safetensors'
    }
    
    base_path = '/workspace/ComfyUI'
    for path, url in models.items():
        full_path = f"{base_path}/{path}"
        print(f"Downloading {path}...")
        download_file(url, full_path)

def handle_antelope():
    zip_url = "https://huggingface.co/MonsterMMORPG/tools/resolve/main/antelopev2.zip"
    zip_path = "/tmp/antelopev2.zip"
    extract_path = "/workspace/ComfyUI/models/insightface/models/antelopev2"
    
    download_file(zip_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    os.remove(zip_path)

print("Starting FLUX workflow setup...")
check_versions()
setup_directories()
install_dependencies()
download_models()
handle_antelope()
print("Setup complete!")